In [2]:
import openai
import pandas as pd
import random

In [13]:
# api_key를 입력하세요.
openai.api_key = 

In [14]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0.1, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message.content

In [15]:
# 데이터 불러오기
df = pd.read_csv("./data/df_cluster.csv")
df.head()

,url,title,date,content,nouns,cluster_dbscan,cluster
0,https://n.news.naver.com/mnews/article/001/001...,"이스라엘, 가자 '구호트럭 참사' 원인은 ""공격 아닌 압사""",2024-03-03,"""압사 방지 위한 경고 사격 후 약탈자들이 위협해 대응 사격"" 가자시티 구호 트럭...","['압사', '방지', '경고', '사격', '약탈', '위협', '사격', '시티...",0,NaN
1,https://n.news.naver.com/mnews/article/008/000...,"與윤재옥 ""김영주 부의장 입당, 우리 당 시야 넓히는 데 도움""",2024-03-03,"[the300] 현역 교체 비율 낮다는 지적에 ""국민들 눈높이에서 생각해보고 개선...","['현역', '교체', '비율', '지적', '눈높이', '개선', '개선', '윤...",1,11.0
2,https://n.news.naver.com/mnews/article/016/000...,빅5 대형병원도 응급실 '빨간불'…전임의 이탈 촉각,2024-03-03,"상급종합병원, 환자 더 줄일 듯정부 ""오늘까지 돌아오면 선처""거리 나온 의사들 ""...","['상급', '종합병원', '환자', '듯', '선처', '거리', '의사', '의...",2,8.0
3,https://n.news.naver.com/mnews/article/031/000...,"대통령실 ""의대 증원 2천명 변화 없어…비수도권 집중 배치""",2024-03-03,"현장 복귀 안한 전공의 ""법과 원칙에 따른 절차 불가피""성태윤 대통령실 정책실장은...","['복귀', '전공', '원칙', '불가피', '성태', '윤', '대통령실', '...",3,8.0
4,https://n.news.naver.com/mnews/article/001/001...,먹과 두드림으로 기록하는 역사…국보 정림사지 석탑 탁본 뜬다,2024-03-03,"문화재위원회, 총 4건 금석문 탁본 허가…고대사·서예사 연구에 도움 국보 '부여 ...","['문화재', '총', '금석문', '탁본', '허가', '대사', '예사', '연...",4,6.0


In [20]:
score = []
keyword = []
cluster = []

for i in list(set(df['cluster'])):
    if (i == -1) or (pd.isna(i)) or(i == 6): # 내부 거리 가장 컸던 클러스터 추가
        continue
    li_title = []
    titles_list = list(df[df['cluster'] == i]['title'])
    random.shuffle(titles_list)
    for title in titles_list:
        li_title.append(title)
        if (len(li_title) > 90):
            break
    score.append(len(df[df['cluster'] == i]))
    cluster.append(i)
    story = li_title.copy()
    prompt = f"""
    Determine main one keyword that are being discussed in the following text.
    
    Please present keyword by short word phrases.
    
    Please present the answer only in Korean.
    
    Text sample: '''{story}'''
    """
    response = get_completion(prompt)
    keyword.append(response)
    print(f"cluster : {i} number : {len(li_title)} \n ---------------------- \n {response}")

cluster : 0.0 number : 81 
 ---------------------- 
 태아 성 감별
cluster : 2.0 number : 87 
 ---------------------- 
 의사 집회
cluster : 1.0 number : 71 
 ---------------------- 
 이동관
cluster : 3.0 number : 53 
 ---------------------- 
 실거주 의무
cluster : 7.0 number : 91 
 ---------------------- 
 쌍특검법
cluster : 8.0 number : 91 
 ---------------------- 
 의대 증원
cluster : 11.0 number : 58 
 ---------------------- 
 전세사기
cluster : 12.0 number : 91 
 ---------------------- 
 운전자
cluster : 13.0 number : 91 
 ---------------------- 
 성관계
cluster : 14.0 number : 82 
 ---------------------- 
 수은법
cluster : 15.0 number : 91 
 ---------------------- 
 총선
cluster : 16.0 number : 57 
 ---------------------- 
 임대차 계약갱신청구권
cluster : 17.0 number : 62 
 ---------------------- 
 '돈봉투'
cluster : 18.0 number : 91 
 ---------------------- 
 압수수색
cluster : 19.0 number : 91 
 ---------------------- 
 전공의
cluster : 20.0 number : 91 
 ---------------------- 
 딥페이크
cluster : 21.0 number : 56 
 ---------------------- 
 

In [21]:
# 키워드 정리 : 키워드 여러 개 있는 것 삭제 -> 이 과정 필요없도록 만들어야 함
keyword_df = pd.DataFrame({"keyword" : keyword, "cluster" : cluster, "score" : score})
keyword_df = keyword_df.sort_values("score", ascending = False).reset_index(drop = True)

for i in range(len(keyword_df)):
    if "," in keyword_df.loc[i, "keyword"]:
        keyword_df.drop(i, axis = 0, inplace = True)

keyword_df = keyword_df.reset_index(drop = True)

In [22]:
li_keyword = []
for i in range(len(df)):
    k = len(li_keyword)
    for j in range(len(keyword_df)):
        if df.loc[i, 'cluster'] == keyword_df.loc[j, 'cluster']:
            li_keyword.append(keyword_df.loc[j, 'keyword'])
    k_new = len(li_keyword)
    if k == k_new:
        li_keyword.append('notaissue')
            
df['keyword'] = li_keyword

In [23]:
keyword_df.to_csv("./data/keyword_rank.csv", index = False)
pd.read_csv("./data/keyword_rank.csv")

,keyword,cluster,score
0,딥페이크,20.0,487
1,성관계,13.0,331
2,전공의,19.0,258
3,의대 증원,8.0,244
4,총선,15.0,228
5,쌍특검법,7.0,191
6,압수수색,18.0,189
7,운전자,12.0,137
8,선거구 획정안,4.0,117
9,선거구 획정,10.0,106
